In [1]:
import tensorflow as tf
from forward_kinematics import forward_kinematics
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam

In [2]:
def lerp(i, j, k):
    return float((1 - k) * i + j * k)

def invlerp(i, j, k):
    return float((k - i) / (j - i))

# Create data

In [3]:
# Generate 220000 samples
x_samples=[]
y_samples=[]

#for i in range(220000):
for i in range(1):
    T1=np.random.randint(-90,91)
    T2=np.random.randint(-90,91)
    T3=np.random.randint(-90,91)
    T4=np.random.randint(-90,91)        
    fk=forward_kinematics(T1,T2,T3,T4)[:3,3]
    x_samples.append(fk)
    y_samples.append([T1,T2,T3,T4])
    
x_samples=np.array(x_samples)
y_samples=np.array(y_samples,dtype=np.float32)

In [4]:
np.max(x_samples)

411.0

## Data normalization

In [5]:
x_max=np.max(x_samples[:,0])
y_max=np.max(x_samples[:,1])
z_max=np.max(x_samples[:,2])
x_min=np.min(x_samples[:,0])
y_min=np.min(x_samples[:,1])
z_min=np.min(x_samples[:,2])
x_norm_samples=[]
y_norm_samples=[]
#for i in range(220000):
for i in range(1):
    fk=x_samples[i]
    tada=[]
    tada.append(lerp(-1,1,invlerp(x_max, x_min, fk[0])))
    tada.append(lerp(-1,1,invlerp(y_max, y_min, fk[1])))
    tada.append(lerp(-1,1,invlerp(z_max, z_min, fk[2])))
    x_norm_samples.append(tada)
    angles=y_samples[i]
    y_norm_samples.append(angles/90)
x_norm_samples=np.array(x_norm_samples)
y_norm_samples=np.array(y_norm_samples)

C:\Users\salih\AppData\Local\Temp\ipykernel_7536\275508645.py:5: RuntimeWarning: invalid value encountered in scalar divide
  return float((k - i) / (j - i))


# Split data

In [6]:
X_train = x_norm_samples[:200000]
X_val = x_norm_samples[200000:]

In [7]:
y_train = y_norm_samples[:200000]
y_val = y_norm_samples[200000:]

# Create model

In [8]:
model_1=tf.keras.Sequential([
    layers.Dense(32, activation='relu'),
    layers.Dense(64,activation='relu'),
    layers.Dense(64,activation='relu'),
    layers.Dense(128,activation='relu'),
    layers.Dense(128,activation='relu'),
    layers.Dense(64,activation='relu'),
    layers.Dense(64,activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1,activation="linear"),
])

In [10]:
# Compile model
loss = tf.keras.losses.mse
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-2)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=100, restore_best_weights=True)
model_1.compile(loss=loss, optimizer=optimizer, metrics=tf.metrics.RootMeanSquaredError())

In [19]:
train_data =  tf.data.Dataset.from_tensor_slices((X_train, y_train[:,0])).batch(32)
val_data =  tf.data.Dataset.from_tensor_slices((X_val, y_val))

In [21]:
# Train model
model_1.fit(X_train, y_train[:,0], 
            epochs=40,
            validation_data=(X_val,y_val), 
            callbacks=[early_stop])

Epoch 1/40
1/1 [==============================] - 0s 251ms/step - loss: nan - root_mean_squared_error: nan
Epoch 2/40
1/1 [==============================] - 0s 16ms/step - loss: nan - root_mean_squared_error: nan
Epoch 3/40
1/1 [==============================] - 0s 16ms/step - loss: nan - root_mean_squared_error: nan
Epoch 4/40
1/1 [==============================] - 0s 16ms/step - loss: nan - root_mean_squared_error: nan
Epoch 5/40
1/1 [==============================] - 0s 16ms/step - loss: nan - root_mean_squared_error: nan
Epoch 6/40
1/1 [==============================] - 0s 16ms/step - loss: nan - root_mean_squared_error: nan
Epoch 7/40
1/1 [==============================] - 0s 16ms/step - loss: nan - root_mean_squared_error: nan
Epoch 8/40
1/1 [==============================] - 0s 16ms/step - loss: nan - root_mean_squared_error: nan
Epoch 9/40
1/1 [==============================] - 0s 16ms/step - loss: nan - root_mean_squared_error: nan
Epoch 10/40
1/1 [============================

In [119]:
#model_1.evaluate(X_val,y_val[:,0])

625/625 [==============================] - 1s 670us/step - loss: 0.3413 - root_mean_squared_error: 0.5842


[0.3413074314594269, 0.5842152237892151]

In [106]:
x_samples[1], y_train[1]*90

(array([-165.,  159., -104.]), array([ 44., -85., -31., -49.], dtype=float32))

In [5]:
T1=44
T2=85
T3=31
T4=49
fk=forward_kinematics(T1,T2,T3,T4)[:3,3]
fk

array([ 165., -159., -104.])

In [108]:
np.min(X_train)

-1.0

In [109]:
ala=[]
ala.append(lerp(-1,1,invlerp(x_max, x_min, fk[0])))
ala.append(lerp(-1,1,invlerp(y_max, y_min, fk[1])))
ala.append(lerp(-1,1,invlerp(z_max, z_min, fk[2])))
ala

[0.44959128065395104, -0.4340136054421768, 0.7943037974683544]

In [110]:
output=model_1.predict(np.array([ala]))
output

1/1 [==============================] - 0s 73ms/step


array([[-0.03899958, -0.03899958, -0.03899958, -0.03899958]],
      dtype=float32)

In [111]:
out=np.round(output*90)[0]
out

array([-4., -4., -4., -4.], dtype=float32)

In [112]:
fk=forward_kinematics(out[0],out[1],out[2],out[3])[:3,3]
fk

array([-56.,  -4., 458.])

In [ ]:
fk[0] = lerp(0,1,invlerp(463., -463., fk[0]))
fk[1] = lerp(0,1,invlerp(368., -368., fk[1]))
fk[2] = lerp(0,1,invlerp(368., -368., fk[2]))

In [ ]:
fk

In [ ]:
X_train[:5]

In [ ]:
np.max(y_train[:,1])

In [ ]:
X_train[:,0]

In [100]:
y_train

array([[-0.5       , -0.84444445, -0.43333334, -0.11111111],
       [ 0.4888889 , -0.9444444 , -0.34444445, -0.54444444],
       [-0.98888886,  0.02222222,  0.46666667,  0.5777778 ],
       ...,
       [ 0.14444445, -0.3888889 , -0.9222222 ,  0.3888889 ],
       [ 0.25555557,  0.4888889 , -0.6       , -0.11111111],
       [-0.06666667,  0.2       , -0.6666667 ,  0.87777776]],
      dtype=float32)

In [102]:
y_train[:,0]

array([-0.5       ,  0.4888889 , -0.98888886, ...,  0.14444445,
        0.25555557, -0.06666667], dtype=float32)

In [ ]:
invlerp(-90.0, 90.0, 0)

# Try 2 way

# Create data

In [22]:
# Generate 220000 samples
x_samples=[]
y_samples=[]

for i in range(220000):
    T1=np.random.randint(-0,91)
    T2=np.random.randint(-0,91)
    T3=np.random.randint(-0,91)
    T4=np.random.randint(-0,91)        
    fk=forward_kinematics(T1,T2,T3,T4)[:3,3]
    r=np.sqrt(np.square(fk[0])+np.square(fk[1]))
    #x_samples.append([r, fk[2]-95.])
    x_samples.append([r, fk[2]-95.,T2/90])
    #y_samples.append([T2,T3,T4])
    y_samples.append([T3,T4])

    
x_samples=np.array(x_samples)
y_samples=np.array(y_samples,dtype=np.float32)

In [23]:
x_samples[:,0]

array([284.25516706, 217.41205118, 311.9775633 , ..., 212.50882335,
       244.64055265,   5.09901951])

## Data normalization

In [24]:
r_max=np.max(x_samples[:,0])
r_min=np.min(x_samples[:,0])
z_max=np.max(x_samples[:,1])
z_min=np.min(x_samples[:,1])
x_norm_samples=[]
y_norm_samples=[]
for i in range(220000):
    fk=x_samples[i]
    tada=[]
    tada.append(lerp(1,0,invlerp(r_max, r_min, fk[0])))
    tada.append(lerp(1,0,invlerp(z_max, z_min, fk[1])))
    tada.append(fk[2]/90)
    x_norm_samples.append(tada)
    angles=y_samples[i]
    y_norm_samples.append(angles/90)
x_norm_samples=np.array(x_norm_samples)
y_norm_samples=np.array(y_norm_samples)

In [25]:
y_norm_samples

array([[0.35555556, 0.08888889],
       [0.35555556, 0.76666665],
       [0.6       , 0.11111111],
       ...,
       [0.2888889 , 0.9444444 ],
       [0.13333334, 0.4888889 ],
       [0.87777776, 0.9111111 ]], dtype=float32)

# Split data

In [104]:
X_train = x_norm_samples[:200000]
X_val = x_norm_samples[200000:]

In [105]:
y_train = y_norm_samples[:200000]
y_val = y_norm_samples[200000:]

In [106]:
for i in range(200000):
    if(X_train[i][0]==1):
        print(X_train[i])
        print(x_samples[i])
        break;

[1.        0.4193038 0.0108642]
[368.26620806   1.           0.97777778]


In [107]:
np.min(x_norm_samples[:,1])

0.0

# Create model

In [108]:
model_2=tf.keras.Sequential([
    layers.Dense(32, activation='tanh'),

    layers.Dense(64,activation='tanh'),
    layers.Dense(32, activation='tanh'),
    layers.Dense(2,activation="linear"),
])

In [109]:
# Compile model
loss = tf.keras.losses.mae
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-2)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=100, restore_best_weights=True)
model_2.compile(loss=loss, optimizer=optimizer, metrics=tf.metrics.RootMeanSquaredError())

In [110]:
train_data =  tf.data.Dataset.from_tensor_slices((X_train, y_train))#.batch(512)
#val_data =  tf.data.Dataset.from_tensor_slices((X_val, y_val))

In [111]:
# Train model
model_2.fit(X_train,y_train, 
            epochs=50,
            validation_data=(X_val,y_val), 
            callbacks=[early_stop])

Epoch 1/50
6250/6250 [==============================] - 4s 602us/step - loss: 0.1989 - root_mean_squared_error: 0.2423 - val_loss: 0.1961 - val_root_mean_squared_error: 0.2393
Epoch 2/50
6250/6250 [==============================] - 4s 598us/step - loss: 0.1931 - root_mean_squared_error: 0.2363 - val_loss: 0.1888 - val_root_mean_squared_error: 0.2314
Epoch 3/50
6250/6250 [==============================] - 4s 599us/step - loss: 0.1730 - root_mean_squared_error: 0.2125 - val_loss: 0.1687 - val_root_mean_squared_error: 0.2074
Epoch 4/50
6250/6250 [==============================] - 4s 597us/step - loss: 0.1633 - root_mean_squared_error: 0.2009 - val_loss: 0.1633 - val_root_mean_squared_error: 0.2015
Epoch 5/50
6250/6250 [==============================] - 4s 597us/step - loss: 0.1625 - root_mean_squared_error: 0.2004 - val_loss: 0.1631 - val_root_mean_squared_error: 0.2018
Epoch 6/50
6250/6250 [==============================] - 4s 593us/step - loss: 0.1619 - root_mean_squared_error: 0.2000 -

6250/6250 [==============================] - 4s 592us/step - loss: 0.1434 - root_mean_squared_error: 0.1804 - val_loss: 0.1370 - val_root_mean_squared_error: 0.1709
Epoch 48/50
6250/6250 [==============================] - 4s 592us/step - loss: 0.1430 - root_mean_squared_error: 0.1798 - val_loss: 0.1378 - val_root_mean_squared_error: 0.1737
Epoch 49/50
6250/6250 [==============================] - 4s 595us/step - loss: 0.1423 - root_mean_squared_error: 0.1790 - val_loss: 0.1350 - val_root_mean_squared_error: 0.1696
Epoch 50/50
6250/6250 [==============================] - 4s 598us/step - loss: 0.1420 - root_mean_squared_error: 0.1788 - val_loss: 0.1398 - val_root_mean_squared_error: 0.1738


In [77]:
model_2.evaluate(X_val,y_val[:,0])

InvalidArgumentError: Graph execution error:

Detected at node 'SquaredDifference' defined at (most recent call last):
    File "C:\Users\salih\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\salih\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\salih\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\salih\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\salih\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\Users\salih\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\salih\anaconda3\lib\asyncio\base_events.py", line 601, in run_forever
      self._run_once()
    Fil

In [78]:
T1=np.random.randint(0,91)
T2=np.random.randint(0,91)
T3=np.random.randint(0,91)
T4=np.random.randint(0,91)  
fk_true=forward_kinematics(T1,T2,T3,T4)[:3,3]
fk_true, [T1,T2,T3,T4]

(array([ 13.,  -0., -98.]), [1, 88, 66, 74])

In [79]:
predict=[]
r=np.sqrt(np.square(fk_true[0])+np.square(fk_true[1]))
predict=[r, fk[2]-95.]

In [80]:
ala=[]
ala.append(lerp(1,0,invlerp(r_max, r_min, predict[0])))
ala.append(lerp(1,0,invlerp(z_max, z_min, predict[1])))
ala.append(T2/90)
ala

[0.035300550839106104, 0.26865330520393815, 0.9777777777777777]

In [81]:
output=model_2.predict(np.array([ala]))
output

1/1 [==============================] - 0s 56ms/step


array([[0.0255692, 0.0610252]], dtype=float32)

In [82]:
out=np.round(output*90)[0]
out

array([2., 5.], dtype=float32)

In [83]:
fk=forward_kinematics(T1,T2,out[0],out[1])[:3,3]
fk, fk_true

(array([367.,  -6.,  83.]), array([ 13.,  -0., -98.]))